# What assessments are the Instructors using?
* When are things due?
* How long are things open?
* How many points are they worth?
* How many students does this affect?


The query:
```sql
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME,
       emails,
       userNames,
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open,
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       POSSIBLE,
       students,
       CNTHNDLR_HANDLE,
       RULE_TYPE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       MULTIPLE_ATTEMPTS,
       force_completion_IND,
       time_limit,
       questions
FROM (SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       gm.POSSIBLE,
       students.students,
       cc.CNTHNDLR_HANDLE,
       ar.TITLE as ar_title,
       ar.RULE_TYPE,
       ar.PK1 as ar_pk1,
       cc.DTCREATED,
       cc.DTMODIFIED,
       gm.MULTIPLE_ATTEMPTS,
       ca.FORCE_COMPLETION_IND,
       ca.time_limit,
       questions.questions,
       instructors.emails emails,
       instructors.userNames userNames,
       rank() over (partition by ar.PK1 order by ac.CRIT_TYPE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1
    left JOIN (select cu.CRSMAIN_PK1,
                      listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
                      listagg(u.user_id, ';') WITHIN GROUP (order by u.email) userNames
        from bb_bb60.COURSE_USERS cu join bb_bb60.USERS u on cu.USERS_PK1 = u.PK1
        WHERE (role='P' OR role='T') and
              cu.ROW_STATUS = 0
              group by cu.CRSMAIN_PK1) instructors on instructors.crsmain_pk1 = cm.pk1
    LEFT JOIN (SELECT qti1.PK1 pk1, COUNT(qti3.pk1) questions
        from bb_bb60.QTI_ASI_DATA qti1
        JOIN bb_bb60.QTI_ASI_DATA qti2 on qti1.PK1 = qti2.PARENT_PK1
        JOIN bb_bb60.QTI_ASI_DATA qti3 on qti2.PK1 = qti3.PARENT_PK1
    WHERE qti1.PARENT_PK1 IS NULL AND QTI3.BBMD_ASI_TYPE = 3
    GROUP BY qti1.PK1) questions on gm.QTI_ASI_DATA_PK1 = questions.pk1
     WHERE cm.COURSE_ID LIKE '4202%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
  and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule')
  and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL))
    WHERE rnk = 1
ORDER BY DUE_DATE;
```

In [2]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

In [10]:
query = """
SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    
    WHERE cm.COURSE_ID LIKE '4206-%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
      and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL)
    ORDER BY gm.DUE_DATE
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,start_date,end_date,due_date,pk1,title,available_ind,cnthndlr_handle,dtcreated,dtmodified
0,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
1,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,2020-07-13 05:00:00,2020-07-13 09:00:00,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
2,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
3,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,2020-07-13 16:00:00,2020-07-13 18:00:00,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
4,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38


### Problems
There's multiple assignments? Same PK - but some have start and some don't? WHAT IS HAPPENING?
Availability rules are happening.  This is useful for an instructor trying to give more time to a student or group... but not to a developer trying to pull out the 'default' availability.
* Why doesn't this work in csv?
* Which of these assignments are real?
* How long are these open?

In [7]:
query = """
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME, --removed commas from course names
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open, -- added calculation
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       CNTHNDLR_HANDLE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       rnk  -- to use this in the where clause, it's easiest to pull it directly out of a subquery in Oracle
FROM(SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED,
       ar.TITLE artitle,
       rank() over (partition by cc.PK1 order by ac.CRIT_TYPE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    
    WHERE cm.COURSE_ID LIKE '4206-%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
      and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL)
      and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule'))
WHERE rnk = 1
ORDER BY DUE_DATE
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,start_date,end_date,hours_open,due_date,title,available_ind,cnthndlr_handle,pk1,dtcreated,dtmodified,artitle,rnk
0,4206-75610,2020Summer-DSCI 302 Quantitative Methods and D...,2020-07-09 14:00:00,2020-07-15 14:00:00,144.0,2020-07-14 14:00:00,HW - Covering Material from July 9,Y,resource/x-bb-assignment,8409473,2020-07-09 11:44:14,2020-07-09 11:44:14,None,1
1,4206-70953,2020Summer-MATH 2 Intermediate Mathematics LEC,2020-07-14 09:00:00,2020-07-14 16:00:00,7.0,2020-07-14 16:00:00,Quiz 5.1/5.2,Y,resource/x-bb-asmt-test-link,8288181,2020-05-11 16:25:03,2020-07-14 08:46:23,Test Option Default Rule,1
2,4206-76322,2020Summer-SPED 854 Family and Interprofession...,NaT,NaT,NaN,2020-07-14 16:30:00,Reading Reflection J,Y,resource/x-bb-assignment,8330213,2020-05-28 11:28:47,2020-07-10 14:27:56,None,1
3,4206-77564,2020Summer-ABSC 851 Ethical Legal and Profes...,NaT,2020-07-14 17:00:00,NaN,2020-07-14 17:00:00,Week 5 Assignment - the Weiss Presentation,Y,resource/x-bb-assignment,8289699,2020-05-12 16:04:27,2020-07-08 11:29:06,None,1
4,4206-77564,2020Summer-ABSC 851 Ethical Legal and Profes...,NaT,2020-07-14 17:00:00,NaN,2020-07-14 17:00:00,Week 5 Reading Summaries,Y,resource/x-bb-assignment,8289698,2020-05-12 16:04:27,2020-07-08 11:29:06,None,1


### Problems
* Are these real tests? Or just an instructor practicing?
* Do these tests have concerning settings?
* How many students are affected? 
* Who teaches these courses?


In [13]:
query = """
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME,
       emails,
       userNames,
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open,
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       POSSIBLE,
       students,
       CNTHNDLR_HANDLE,
       RULE_TYPE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       MULTIPLE_ATTEMPTS, --Multiple attempts are nice
       force_completion_IND, --Force completion is very concerning
       time_limit, --Time limits can be concerning
       questions
FROM (SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       gm.POSSIBLE,
       students.students,
       cc.CNTHNDLR_HANDLE,
       ar.TITLE as ar_title,
       ar.RULE_TYPE,
       ar.PK1 as ar_pk1,
       cc.DTCREATED,
       cc.DTMODIFIED,
       gm.MULTIPLE_ATTEMPTS,
       ca.FORCE_COMPLETION_IND,
       ca.time_limit,
       questions.questions,
       instructors.emails emails,
       instructors.userNames userNames,
       rank() over (partition by ar.PK1 order by ac.CRIT_TYPE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
        
        ---Subquery to get a count of available students
        left join (select crsmain_pk1, count(pk1) students
        from bb_bb60.COURSE_USERS
        where role='S'
        and row_status = 0
        group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1
        
        ---Subquery to get an aggregated list of userIds and emails.
        left JOIN (select cu.CRSMAIN_PK1,
                      listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
                      listagg(u.user_id, ';') WITHIN GROUP (order by u.email) userNames
        from bb_bb60.COURSE_USERS cu join bb_bb60.USERS u on cu.USERS_PK1 = u.PK1
        WHERE (role='P' OR role='T') and
              cu.ROW_STATUS = 0
              group by cu.CRSMAIN_PK1) instructors on instructors.crsmain_pk1 = cm.pk1
        
        ---Subquery to get a count of questions (if a test)
        LEFT JOIN (SELECT qti1.PK1 pk1, COUNT(qti3.pk1) questions -- VERY complicated way to pull questions on a test
            from bb_bb60.QTI_ASI_DATA qti1
            JOIN bb_bb60.QTI_ASI_DATA qti2 on qti1.PK1 = qti2.PARENT_PK1
            JOIN bb_bb60.QTI_ASI_DATA qti3 on qti2.PK1 = qti3.PARENT_PK1
        WHERE qti1.PARENT_PK1 IS NULL AND QTI3.BBMD_ASI_TYPE = 3 
        GROUP BY qti1.PK1) questions on gm.QTI_ASI_DATA_PK1 = questions.pk1
     
     WHERE cm.COURSE_ID LIKE '4202%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
  and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule')
  and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL))
    WHERE rnk = 1
ORDER BY DUE_DATE
"""
upcoming_df = pd.read_sql(query, con=engine)

In [14]:
upcoming_df.head()

,course_id,course_name,emails,usernames,start_date,end_date,hours_open,due_date,title,available_ind,...,students,cnthndlr_handle,rule_type,pk1,dtcreated,dtmodified,multiple_attempts,force_completion_ind,time_limit,questions
0,4202-57575,2020Spring-CLSX 350 Modern Themes Ancient Mod...,scioli@ku.edu,scioli,2020-07-22 12:00:00,NaT,NaN,2020-07-28 23:59:00,Final Exam,N,...,16.0,resource/x-bb-assignment,None,8285264,2020-05-08 17:00:11,2020-06-20 20:46:58,0,None,NaN,NaN
1,4202-43900,2020Spring-BIOL 247 Principles of Human Physio...,dswartzlander@ku.edu;towineinger@ku.edu,d884s250;t062w968,2020-08-15 12:00:00,2020-08-22 09:00:00,165.0,2020-08-22 09:00:00,Kidney Function Pre-Lab Quiz,Y,...,14.0,resource/x-bb-asmt-test-link,None,8077861,2020-01-22 09:33:04,2020-04-19 20:34:45,0,N,NaN,5.0
2,4202-43900,2020Spring-BIOL 247 Principles of Human Physio...,dswartzlander@ku.edu;towineinger@ku.edu,d884s250;t062w968,2020-08-15 12:00:00,2020-08-22 09:00:00,165.0,2020-08-22 09:00:00,Fitness and Health Pre-Lab Quiz,Y,...,14.0,resource/x-bb-asmt-test-link,None,8077862,2020-01-22 09:33:04,2020-04-19 20:35:05,0,N,NaN,4.0
3,4202-55747,2020Spring-PSYC 333 Child Development LEC,arpope@ku.edu;bbadmin@ku.edu;jfulton0922@ku.ed...,a026p699;administrator;j551f302;_bbsupport,NaT,NaT,NaN,2020-09-08 23:59:00,Team topic interest,N,...,28.0,resource/x-bb-assignment,None,8061916,2020-01-21 00:37:56,2020-02-11 20:23:14,0,None,NaN,NaN
4,4202-55747,2020Spring-PSYC 333 Child Development LEC,arpope@ku.edu;bbadmin@ku.edu;jfulton0922@ku.ed...,a026p699;administrator;j551f302;_bbsupport,NaT,NaT,NaN,2020-09-09 21:00:00,Week 3 In class assignment,N,...,28.0,resource/x-bb-assignment,None,8061928,2020-01-21 00:37:56,2020-01-21 16:00:26,1,None,NaN,NaN


In [15]:
len(upcoming_df)

9605

In [16]:
max(upcoming_df['due_date'])

Timestamp('2030-03-30 23:55:00')

In [18]:
upcoming_df[upcoming_df['due_date'] > pd.to_datetime('2020-09-01')]

,course_id,course_name,emails,usernames,start_date,end_date,hours_open,due_date,title,available_ind,...,students,cnthndlr_handle,rule_type,pk1,dtcreated,dtmodified,multiple_attempts,force_completion_ind,time_limit,questions
3,4202-55747,2020Spring-PSYC 333 Child Development LEC,arpope@ku.edu;bbadmin@ku.edu;jfulton0922@ku.ed...,a026p699;administrator;j551f302;_bbsupport,NaT,NaT,NaN,2020-09-08 23:59:00,Team topic interest,N,...,28.0,resource/x-bb-assignment,None,8061916,2020-01-21 00:37:56,2020-02-11 20:23:14,0,None,NaN,NaN
4,4202-55747,2020Spring-PSYC 333 Child Development LEC,arpope@ku.edu;bbadmin@ku.edu;jfulton0922@ku.ed...,a026p699;administrator;j551f302;_bbsupport,NaT,NaT,NaN,2020-09-09 21:00:00,Week 3 In class assignment,N,...,28.0,resource/x-bb-assignment,None,8061928,2020-01-21 00:37:56,2020-01-21 16:00:26,1,None,NaN,NaN
5,4202-48665,2020Spring-PSYC 104 General Psychology LEC,genna.losinski@ku.edu;kadams_18@ku.edu,g479l368;k907a808,NaT,NaT,NaN,2020-09-23 23:59:00,Psychology in Action! submission link,Y,...,47.0,resource/x-bb-assignment,None,8014564,2020-01-16 10:29:58,2020-01-16 10:29:58,3,None,NaN,NaN
6,4202-48655,2020Spring-PSYC 104 General Psychology LEC,dchapa@ku.edu;genna.losinski@ku.edu,d615c138;g479l368,NaT,NaT,NaN,2020-10-02 23:59:00,Submit Research Participation Paper Here,Y,...,80.0,resource/x-bb-assignment,None,7993164,2020-01-14 09:23:05,2020-01-14 20:06:40,3,None,NaN,NaN
7,4202-58271,2020Spring-PSYC 105 General Psychology Honors...,dchapa@ku.edu;genna.losinski@ku.edu,d615c138;g479l368,NaT,NaT,NaN,2020-10-02 23:59:00,Submit Research Participation Paper Here,Y,...,40.0,resource/x-bb-assignment,None,7992718,2020-01-14 09:20:08,2020-01-14 09:20:08,3,None,NaN,NaN
8,4202-52868,2020Spring-ABSC 150 Community Leadership LEC,courtneymoore@ku.edu;d864s644@ku.edu;jomellaw@...,c867m884;d864s644;jomellaw;p185v935,2018-12-28 00:00:00,2020-01-12 23:59:00,9143.98,2020-12-31 23:59:00,Quiz #1,Y,...,21.0,resource/x-bb-asmt-test-link,None,7924869,2019-12-21 09:24:10,2020-01-10 17:57:24,2,Y,90.0,17.0
9,4202-41843,2020Spring-KISW 120 Elementary KiSwahili II LEC,muchira@ku.edu;wawireb@ku.edu,j538m005;b021w602,NaT,NaT,NaN,2021-01-03 23:59:00,Homework 1,Y,...,8.0,resource/x-bb-assignment,None,8106460,2020-01-31 16:31:16,2020-02-14 13:55:39,1,None,NaN,NaN
10,4202-58766,2020Spring-ANTH 100 General Anthropology LEC,garnettj@ku.edu;lauren_norman@ku.edu,j379g152;l460n488,2021-02-01 00:00:00,2021-02-07 23:59:00,167.98,2021-02-07 23:59:00,Practice Test #1,Y,...,30.0,resource/x-bb-asmt-test-link,None,7975911,2020-01-10 21:18:18,2020-02-04 13:31:01,0,N,45.0,1.0
11,4202-58764,2020Spring-ANTH 100 General Anthropology LEC,garnettj@ku.edu;kdoeden@ku.edu;lauren_norman@k...,j379g152;k390d510;l460n488,2021-02-01 00:00:00,2021-02-07 23:59:00,167.98,2021-02-07 23:59:00,Practice Test #1,Y,...,30.0,resource/x-bb-asmt-test-link,None,8141822,2020-03-02 08:49:18,2020-03-18 13:01:12,0,N,45.0,1.0
12,4202-48665,2020Spring-PSYC 104 General Psychology LEC,genna.losinski@ku.edu;kadams_18@ku.edu,g479l368;k907a808,NaT,NaT,NaN,2021-02-27 23:59:00,Submit Research Participation Paper Here,Y,...,47.0,resource/x-bb-assignment,None,8014540,2020-01-16 10:29:58,2020-01-16 10:29:58,3,None,NaN,NaN


Still some issues, as seen above - but this gives a lot of insight into the assignments coming up.

In [26]:
timedf = upcoming_df.groupby([upcoming_df['due_date'].dt.date]).count()['title']

In [27]:
timedf

due_date
2020-07-28    1
2020-08-22    2
2020-09-08    1
2020-09-09    1
2020-09-23    1
2020-10-02    2
2020-12-31    1
2021-01-03    1
2021-02-07    2
2021-02-27    1
2021-03-05    1
2021-03-13    1
2021-04-03    2
2021-04-04    1
2021-04-12    1
2021-05-12    1
2021-05-15    1
2029-02-13    1
2030-03-30    1
Name: title, dtype: int64